In [21]:
import pandas as pd
import sqlite3

# Exercise 02. Join

## Task 1. Database Connection

In [22]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## Task 2. Create Datamart Table

In [23]:
query = """
CREATE TABLE IF NOT EXISTS datamart AS
SELECT 
    c.uid,
    c.labname,
    MIN(c.timestamp) as first_commit_ts,
    MIN(p.datetime) as first_view_ts
FROM checker c
LEFT JOIN pageviews p ON c.uid = p.uid
WHERE c.status = 'ready'
    AND c.numTrials = 1
    AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname
"""
conn.execute(query)
conn.commit()

In [24]:
datamart = pd.io.sql.read_sql("SELECT * FROM datamart", conn, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## Task 3. Split into Test and Control

In [25]:
test = datamart[datamart['first_view_ts'].notna()].copy()
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [26]:
control = datamart[datamart['first_view_ts'].isna()].copy()
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [27]:
control['first_view_ts'] = control['first_view_ts'].fillna(test['first_view_ts'].mean())

In [ ]:
test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)
conn.commit()

81

In [30]:
conn.close()